In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain.output_parsers.json import SimpleJsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI

In [3]:
model = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.0)

In [4]:
json_prompt = PromptTemplate.from_template(
    "Return a JSON object with an `answer` key that answers the following question: {question}"
)
json_parser = SimpleJsonOutputParser()
json_chain = json_prompt | model | json_parser

# Invoke without streaming

In [7]:
json_chain.invoke({"question": "Who invented the microscope?"})

{'answer': 'Antonie van Leeuwenhoek'}

# Invoke with streaming
While all parsers support the streaming interface, only certain parsers can stream through partially parsed objects, since this is highly dependent on the output type. Parsers which cannot construct partial objects will simply yield the fully parsed output.

The SimpleJsonOutputParser for example can stream through partial outputs:


In [5]:
list(json_chain.stream({"question": "Who invented the microscope?"}))

[{},
 {'answer': ''},
 {'answer': 'Ant'},
 {'answer': 'Anton'},
 {'answer': 'Antonie'},
 {'answer': 'Antonie van'},
 {'answer': 'Antonie van Lee'},
 {'answer': 'Antonie van Leeu'},
 {'answer': 'Antonie van Leeuwen'},
 {'answer': 'Antonie van Leeuwenho'},
 {'answer': 'Antonie van Leeuwenhoek'}]

# While the PydanticOutputParser cannot:


test this with pydantic output parser


In [4]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import OpenAI

# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # You can add custom validation logic easily with Pydantic.
    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Badly formed question!")
        return field
    
# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)    

# And a query intended to prompt a language model to populate the data structure.

chain = prompt | model | parser
chain.invoke({"query": "Tell me a joke."})

Joke(setup='Why did the tomato turn red?', punchline='Because it saw the salad dressing!')

In [5]:
list(chain.stream({"query": "Tell me a joke."}))

OutputParserException: Invalid json output: 

<generator object RunnableSequence.stream at 0x000002B0BF231540>